In [1]:
import numpy as np
import matplotlib.pyplot as plt
from util import SimpleGA, CMAES, PEPG, OpenES
import cma
import random
from scipy.special import softmax
import collections
import plotly.graph_objects as go

In [14]:
0.5-2.028*0.06, 5/(0.5-2.028*0.06)

(0.37832, 13.216324804398393)

In [8]:
def charging_schedule(y, balanced_factor, P=10, T=48, N=100, B=6, maxu=0.5, beta=0, method='ES', verbose=False):
    balanced_factor = balanced_factor
    method = method

    stay = 0
    y = softmax(y)
    n = y * N
    x_init, final_energy = 0.2*B*n, 0.9*B
    x_step = np.copy(x_init)
    #print(n)
    for k in range(T):
        u = np.zeros(T)
        V = [i for i in range(k+1) if x_step[i] <= final_energy*n[i] - 1e-3]
        totalev = np.sum([n[i] for i in V])
        stay += totalev
        if method == 'fix': budget = P
        for i in V:
            if method == 'ES':
                u[i] = min(min(P/totalev*n[i], n[i]*maxu), final_energy*n[i] - x_step[i])
            elif method == 'fix':
                u[i] = min(n[i]*maxu, final_energy*n[i] - x_step[i])
                if budget < u[i]: 
                    u[i] = budget
                    break
                budget -= u[i]
        x_step = x_step + u
    energy = np.sum(x_step-x_init)
    if verbose:
        print(energy, stay)
    return -(energy - balanced_factor*stay/N)